This notebook was made to merge the new dataset extracted with the previous notebooks with the dataset available at the [GitHub repository](https://github.com/odenizgiz/Podcasts-Data) where the script was found. Moreover, some additional work was necesarry in order to update each item with current description/cover image.

In [ ]:
%%capture
!pip install pattern3

In [ ]:
import pandas as pd
df_old = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/df_popular_podcasts_old.csv')
df_old

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,"Selling the Couch with Melvin Varghese, Ph.D. ...",http://is2.mzstatic.com/image/thumb/Music71/v4...,"['1410', '26', '1321', '1307', '1315']",129,"[37.7, 36.18, 30.83, 30.87, 32.75, 36.15, 33.8...",https://itunes.apple.com/us/podcast/selling-co...,http://sellingthecouch.libsyn.com/rss,http://www.SellingTheCouch.com,Selling the Couch is the #1 podcast for aspiri...
1,Susan Gregg's Food for the Soul,http://is5.mzstatic.com/image/thumb/Music62/v4...,"['1444', '26', '1314', '1307', '1324', '1443',...",50,0,https://itunes.apple.com/us/podcast/susan-greg...,http://www.susangregg.com/feed/podcast,http://www.susangregg.com,You can feel safe and be happy no matter what ...
2,Field Days,http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1474', '26', '1325']",58,"[17.13, 21.95, 23.48, 22.35, 33.47, 42.75, 26....",https://itunes.apple.com/us/podcast/field-days...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/field-days,The Michigan Department of Corrections - Field...
3,Card Talk: The World's Greatest Magic: the Gat...,http://is1.mzstatic.com/image/thumb/Music6/v4/...,"['1461', '26', '1323']",36,"[97.62, 94.87, 94.35, 74.22, 78.5, 84.43, 76.8...",https://itunes.apple.com/us/podcast/card-talk-...,http://taltcast.libsyn.com/rss,http://cardtalk.libsyn.com,Sometime Magic Pro GCB & semi-competitive Magi...
4,The UAV Digest,http://is3.mzstatic.com/image/thumb/Music62/v4...,"['1455', '26', '1323', '1318']",50,"[29.02, 33.55, 30.47, 31.43, 33.5, 33.98, 33.0...",https://itunes.apple.com/us/podcast/the-uav-di...,http://theuavdigest.com/feed/podcast/,http://theuavdigest.com,The UAV Digest is an audio podcast devoted to ...
...,...,...,...,...,...,...,...,...,...
10150,"MTG Pro Tutor - Insights, Tips & Advice from M...",http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1461', '26', '1323', '1460']",229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...
10151,National CASA Podcast,http://is4.mzstatic.com/image/thumb/Music6/v4/...,"['1475', '26', '1325', '1304', '1468']",100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op..."
10152,CAR DISCOVERY｜PodcastLife,http://is4.mzstatic.com/image/thumb/Music4/v4/...,"['1454', '26', '1323', '1460', '1324', '1320',...",54,"[2.72, 2.72, 2.72, 2.55, 2.55, 2.55, 2.55, 2.5...",https://itunes.apple.com/us/podcast/car-discov...,http://feeds2.feedburner.com/CarDiscovery,http://www.dandl.co.jp/car-discovery/,【www.adelcars.com】アデル・カーズ株式会社提供番組：CAR DISCOVER...
10153,Tottenham Hotspur Family,http://is1.mzstatic.com/image/thumb/Music128/v...,"['1467', '26', '1316']",112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...


In [ ]:
df_new = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/df_popular_podcasts_AR.csv')
df_new

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Wilderness', 'Podcasts', 'Sports', 'Leisure'...",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed
1,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['History', 'Podcasts', 'Society & Culture']",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill..."
2,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['Music Commentary', 'Podcasts', 'Music']",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ..."
3,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Fashion & Beauty', 'Podcasts', 'Arts', 'Heal...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...
4,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Comedy Fiction', 'Podcasts', 'Fiction']",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p..."
...,...,...,...,...,...,...,...,...,...,...,...
23005,"Look Out, Sir!","Look Out, Sir! Warhammer 40k Podcast",https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Games', 'Podcasts', 'Leisure', 'Hobbies']",Games,0,0,https://podcasts.apple.com/us/podcast/look-out...,https://feeds.buzzsprout.com/195242.rss,0,"The Look Out, Sir! is a fortnightly (sometimes..."
23006,Gen-Z Media | Wondery,Treasure Island 2020,https://is3-ssl.mzstatic.com/image/thumb/Podca...,"['Drama', 'Podcasts', 'Fiction']",Drama,0,0,https://podcasts.apple.com/us/podcast/treasure...,https://rss.art19.com/treasure-island-2020,0,James Hawkins helps his mom run a motel in mod...
23007,Maximum Fun,Triple Click,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Technology', 'Podcasts', 'Society & Culture']",Technology,0,0,https://podcasts.apple.com/us/podcast/triple-c...,https://feeds.simplecast.com/6WD3bDj7,0,Video games: They’re serious business! Except ...
23008,"Bianca A. Solorzano, M.Ed.",Montessori Babies,https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Education for Kids', 'Podcasts', 'Kids & Fam...",Education for Kids,0,0,https://podcasts.apple.com/us/podcast/montesso...,https://feeds.buzzsprout.com/1443274.rss,0,Current research shows that the first couple y...


In [ ]:
merge = df_old.assign(merge=df_old.Name.isin(df_new.Name))
merge = merge[merge['merge'] == False]
merge

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description,merge
0,"Selling the Couch with Melvin Varghese, Ph.D. ...",http://is2.mzstatic.com/image/thumb/Music71/v4...,"['1410', '26', '1321', '1307', '1315']",129,"[37.7, 36.18, 30.83, 30.87, 32.75, 36.15, 33.8...",https://itunes.apple.com/us/podcast/selling-co...,http://sellingthecouch.libsyn.com/rss,http://www.SellingTheCouch.com,Selling the Couch is the #1 podcast for aspiri...,False
1,Susan Gregg's Food for the Soul,http://is5.mzstatic.com/image/thumb/Music62/v4...,"['1444', '26', '1314', '1307', '1324', '1443',...",50,0,https://itunes.apple.com/us/podcast/susan-greg...,http://www.susangregg.com/feed/podcast,http://www.susangregg.com,You can feel safe and be happy no matter what ...,False
2,Field Days,http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1474', '26', '1325']",58,"[17.13, 21.95, 23.48, 22.35, 33.47, 42.75, 26....",https://itunes.apple.com/us/podcast/field-days...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/field-days,The Michigan Department of Corrections - Field...,False
3,Card Talk: The World's Greatest Magic: the Gat...,http://is1.mzstatic.com/image/thumb/Music6/v4/...,"['1461', '26', '1323']",36,"[97.62, 94.87, 94.35, 74.22, 78.5, 84.43, 76.8...",https://itunes.apple.com/us/podcast/card-talk-...,http://taltcast.libsyn.com/rss,http://cardtalk.libsyn.com,Sometime Magic Pro GCB & semi-competitive Magi...,False
5,Buddhist Meditations,http://is2.mzstatic.com/image/thumb/Music7/v4/...,"['1438', '26', '1314']",82,"[21.82, 53.8, 77.08, 78.45, 54.4, 33.8, 16.92,...",https://itunes.apple.com/us/podcast/buddhist-m...,http://feeds.soundcloud.com/users/soundcloud:u...,http://what-buddha-said.net,"Early Buddhist Monk\nYour kind pilot, doctor, ...",False
...,...,...,...,...,...,...,...,...,...,...
10150,"MTG Pro Tutor - Insights, Tips & Advice from M...",http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1461', '26', '1323', '1460']",229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...,False
10151,National CASA Podcast,http://is4.mzstatic.com/image/thumb/Music6/v4/...,"['1475', '26', '1325', '1304', '1468']",100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op...",False
10152,CAR DISCOVERY｜PodcastLife,http://is4.mzstatic.com/image/thumb/Music4/v4/...,"['1454', '26', '1323', '1460', '1324', '1320',...",54,"[2.72, 2.72, 2.72, 2.55, 2.55, 2.55, 2.55, 2.5...",https://itunes.apple.com/us/podcast/car-discov...,http://feeds2.feedburner.com/CarDiscovery,http://www.dandl.co.jp/car-discovery/,【www.adelcars.com】アデル・カーズ株式会社提供番組：CAR DISCOVER...,False
10153,Tottenham Hotspur Family,http://is1.mzstatic.com/image/thumb/Music128/v...,"['1467', '26', '1316']",112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...,False


In [ ]:
import requests
import pattern3.web as web
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from progressbar import ProgressBar
from bs4 import BeautifulSoup
import re

In [ ]:
list_url = merge['iTunes URL']
list_id = []
for url in list_url:
  try: 
    regex = r"id\d+\?" #only gets id with number in it
    matches = re.search(regex, url, re.MULTILINE)
    if matches != None:
      match = matches.group()
      list_id.append(match)
    else:
      list_id.append(0)
  except:
    list_id.append(0)
merge['IDs'] = list_id

In [ ]:
merge

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description,merge,IDs
0,"Selling the Couch with Melvin Varghese, Ph.D. ...",http://is2.mzstatic.com/image/thumb/Music71/v4...,"['1410', '26', '1321', '1307', '1315']",129,"[37.7, 36.18, 30.83, 30.87, 32.75, 36.15, 33.8...",https://itunes.apple.com/us/podcast/selling-co...,http://sellingthecouch.libsyn.com/rss,http://www.SellingTheCouch.com,Selling the Couch is the #1 podcast for aspiri...,False,id973972218?
1,Susan Gregg's Food for the Soul,http://is5.mzstatic.com/image/thumb/Music62/v4...,"['1444', '26', '1314', '1307', '1324', '1443',...",50,0,https://itunes.apple.com/us/podcast/susan-greg...,http://www.susangregg.com/feed/podcast,http://www.susangregg.com,You can feel safe and be happy no matter what ...,False,id185744728?
2,Field Days,http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1474', '26', '1325']",58,"[17.13, 21.95, 23.48, 22.35, 33.47, 42.75, 26....",https://itunes.apple.com/us/podcast/field-days...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/field-days,The Michigan Department of Corrections - Field...,False,id1065966361?
3,Card Talk: The World's Greatest Magic: the Gat...,http://is1.mzstatic.com/image/thumb/Music6/v4/...,"['1461', '26', '1323']",36,"[97.62, 94.87, 94.35, 74.22, 78.5, 84.43, 76.8...",https://itunes.apple.com/us/podcast/card-talk-...,http://taltcast.libsyn.com/rss,http://cardtalk.libsyn.com,Sometime Magic Pro GCB & semi-competitive Magi...,False,id555097313?
5,Buddhist Meditations,http://is2.mzstatic.com/image/thumb/Music7/v4/...,"['1438', '26', '1314']",82,"[21.82, 53.8, 77.08, 78.45, 54.4, 33.8, 16.92,...",https://itunes.apple.com/us/podcast/buddhist-m...,http://feeds.soundcloud.com/users/soundcloud:u...,http://what-buddha-said.net,"Early Buddhist Monk\nYour kind pilot, doctor, ...",False,id985883609?
...,...,...,...,...,...,...,...,...,...,...,...
10150,"MTG Pro Tutor - Insights, Tips & Advice from M...",http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1461', '26', '1323', '1460']",229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...,False,id1006948082?
10151,National CASA Podcast,http://is4.mzstatic.com/image/thumb/Music6/v4/...,"['1475', '26', '1325', '1304', '1468']",100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op...",False,id336687838?
10152,CAR DISCOVERY｜PodcastLife,http://is4.mzstatic.com/image/thumb/Music4/v4/...,"['1454', '26', '1323', '1460', '1324', '1320',...",54,"[2.72, 2.72, 2.72, 2.55, 2.55, 2.55, 2.55, 2.5...",https://itunes.apple.com/us/podcast/car-discov...,http://feeds2.feedburner.com/CarDiscovery,http://www.dandl.co.jp/car-discovery/,【www.adelcars.com】アデル・カーズ株式会社提供番組：CAR DISCOVER...,False,id152491753?
10153,Tottenham Hotspur Family,http://is1.mzstatic.com/image/thumb/Music128/v...,"['1467', '26', '1316']",112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...,False,id947134514?


In [ ]:
merge.to_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/old_withIDs.csv', index = False)

In [ ]:
#merge = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/old_withIDs.csv')
merge = merge[merge['IDs'] != '0']
merge

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description,merge,IDs
0,"Selling the Couch with Melvin Varghese, Ph.D. ...",http://is2.mzstatic.com/image/thumb/Music71/v4...,"['1410', '26', '1321', '1307', '1315']",129,"[37.7, 36.18, 30.83, 30.87, 32.75, 36.15, 33.8...",https://itunes.apple.com/us/podcast/selling-co...,http://sellingthecouch.libsyn.com/rss,http://www.SellingTheCouch.com,Selling the Couch is the #1 podcast for aspiri...,False,id973972218?
1,Susan Gregg's Food for the Soul,http://is5.mzstatic.com/image/thumb/Music62/v4...,"['1444', '26', '1314', '1307', '1324', '1443',...",50,0,https://itunes.apple.com/us/podcast/susan-greg...,http://www.susangregg.com/feed/podcast,http://www.susangregg.com,You can feel safe and be happy no matter what ...,False,id185744728?
2,Field Days,http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1474', '26', '1325']",58,"[17.13, 21.95, 23.48, 22.35, 33.47, 42.75, 26....",https://itunes.apple.com/us/podcast/field-days...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/field-days,The Michigan Department of Corrections - Field...,False,id1065966361?
3,Card Talk: The World's Greatest Magic: the Gat...,http://is1.mzstatic.com/image/thumb/Music6/v4/...,"['1461', '26', '1323']",36,"[97.62, 94.87, 94.35, 74.22, 78.5, 84.43, 76.8...",https://itunes.apple.com/us/podcast/card-talk-...,http://taltcast.libsyn.com/rss,http://cardtalk.libsyn.com,Sometime Magic Pro GCB & semi-competitive Magi...,False,id555097313?
5,Buddhist Meditations,http://is2.mzstatic.com/image/thumb/Music7/v4/...,"['1438', '26', '1314']",82,"[21.82, 53.8, 77.08, 78.45, 54.4, 33.8, 16.92,...",https://itunes.apple.com/us/podcast/buddhist-m...,http://feeds.soundcloud.com/users/soundcloud:u...,http://what-buddha-said.net,"Early Buddhist Monk\nYour kind pilot, doctor, ...",False,id985883609?
...,...,...,...,...,...,...,...,...,...,...,...
10150,"MTG Pro Tutor - Insights, Tips & Advice from M...",http://is2.mzstatic.com/image/thumb/Music62/v4...,"['1461', '26', '1323', '1460']",229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...,False,id1006948082?
10151,National CASA Podcast,http://is4.mzstatic.com/image/thumb/Music6/v4/...,"['1475', '26', '1325', '1304', '1468']",100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op...",False,id336687838?
10152,CAR DISCOVERY｜PodcastLife,http://is4.mzstatic.com/image/thumb/Music4/v4/...,"['1454', '26', '1323', '1460', '1324', '1320',...",54,"[2.72, 2.72, 2.72, 2.55, 2.55, 2.55, 2.55, 2.5...",https://itunes.apple.com/us/podcast/car-discov...,http://feeds2.feedburner.com/CarDiscovery,http://www.dandl.co.jp/car-discovery/,【www.adelcars.com】アデル・カーズ株式会社提供番組：CAR DISCOVER...,False,id152491753?
10153,Tottenham Hotspur Family,http://is1.mzstatic.com/image/thumb/Music128/v...,"['1467', '26', '1316']",112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...,False,id947134514?


In [ ]:
# Get artwork, feedurl genreIDs from the iTunes API: 
def data_from_API(ID):
    url = 'https://itunes.apple.com/lookup?id=' + ID

    try:
        response = requests.get(url) 
        data = response.json()
        
        if data["resultCount"] != 0: 
            for elem in data["results"]:   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                                                #remove two if here
                            if "artworkUrl600" in elem.keys():
                                artwork = elem["artworkUrl600"]
                            if "feedUrl" in elem.keys(): 
                                rss_feed = elem["feedUrl"]
                            if "genres" in elem.keys():
                                genre_IDs = elem["genres"]
                            if 'collectionName' in elem.keys():
                                title = elem['collectionName']
                            if 'artistName' in elem.keys():
                                author = elem['artistName']
                            if 'primaryGenreName' in elem.keys():
                                prim_genre = elem['primaryGenreName']
                                    
                                    
                                return [artwork, rss_feed, genre_IDs, title, author, prim_genre]
                            
                            else: 
                                return None 

                    else:
                        return None 
                          
        else: 
            return None

    except:
        None 

In [ ]:
merge = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/old_withIDs.csv')
merge = merge[merge['IDs'] != '0']
IDs = merge['IDs'].tolist()
artworks = []
prim_genres = []
for id in IDs:
  matches = re.search(r"\d+", str(id), re.MULTILINE)
  if matches != None:
    try:
      id = matches.group()
      data = data_from_API(id)
      artworks.append(data[0])
      prim_genres.append(data[5])
    except:
      artworks.append(0)
      prim_genres.append(0)
  else:
      artworks.append(0)
      prim_genres.append(0)
merge['Artwork'] = artworks
merge['Primary Genre'] = prim_genres


In [ ]:
merge.to_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/old_updated.csv', index=False)

In [ ]:
new_old = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/old_updated.csv')

In [ ]:
new_old.drop(['merge', 'IDs'], axis= 1, inplace=True)

In [ ]:
new_old[new_old['Primary Genre'] == '0']

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description,merge,IDs,Primary Genre
3,Card Talk: The World's Greatest Magic: the Gat...,0,"['1461', '26', '1323']",36,"[97.62, 94.87, 94.35, 74.22, 78.5, 84.43, 76.8...",https://itunes.apple.com/us/podcast/card-talk-...,http://taltcast.libsyn.com/rss,http://cardtalk.libsyn.com,Sometime Magic Pro GCB & semi-competitive Magi...,False,id555097313?,0
12,Awesome Office: Lead. Create. Inspire.,0,"['1413', '26', '1321', '1324', '1304', '1470']",100,"[50.65, 58.38, 76.55, 49.97, 40.63, 69.43, 71....",https://itunes.apple.com/us/podcast/awesome-of...,http://awesomeoffice.libsyn.com/rss,http://awesomeoffice.org/,"Welcome to the Awesome Office Show, the podcas...",False,id1037532112?,0
17,"Lipstick, High Heels & BS",0,"['1302', '26', '1324']",162,"[62.95, 56.6, 73.78, 44.55, 61.3, 62.28, 58.78...",https://itunes.apple.com/us/podcast/lipstick-h...,http://recordings.talkshoe.com/rss16554.xml,http://www.talkshoe.com/talkshoe/web/tscmd/tc/...,Move over Tom Leykis - make room for the femal...,False,id278832359?,0
18,محبرون,0,"['1450', '26', '1318', '1321']",24,"[15.3, 12.33, 13.82, 10.48, 15.42, 11.83, 18.2...",https://itunes.apple.com/us/podcast/%D9%85%D8%...,http://www.nmcast.net/Podcast/Cast/m7bron/podc...,http://www.newmedia-ar.com,بودكاست صوتي من إنتاج مؤسسة نيو ميديا يأتيك با...,False,id595728099?,0
24,Les chemins de la philosophie,0,"['1443', '26', '1324']",300,"[51.7, 51.98, 52.15, 51.87, 51.77, 51.98, 52.0...",https://itunes.apple.com/us/podcast/les-chemin...,http://radiofrance-podcast.net/podcast09/rss_1...,http://www.radiofrance.fr/,Une rencontre quotidienne entre philosophie et...,False,id390165399?,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8222,White Noise Paranormal Radio,0,"['1479', '26', '1315']",75,"[122.0, 3.0, 122.0, 122.0, 123.0, 123.0, 123.0...",https://itunes.apple.com/us/podcast/white-nois...,http://www.blogtalkradio.com/famously-haunted/...,http://www.blogtalkradio.com/famously-haunted,Next Live Broadcast - Sunday 26th December 201...,False,id315628421?,0
8223,The Runners Round Table,0,"['1467', '26', '1316']",23,0,https://itunes.apple.com/us/podcast/the-runner...,http://feeds.feedburner.com/RunnersRoundtable,http://runnersroundtablepodcast.blogspot.com/,Where runners from around the world share tips...,False,id287953154?,0
8229,"Trading Story: Trading Interviews, Tips & Insp...",0,"['1412', '26', '1321', '1304']",100,"[10.4, 37.72, 44.33, 21.35, 21.85, 51.83, 58.9...",https://itunes.apple.com/us/podcast/trading-st...,http://tradingstory.libsyn.com/rss,http://tradingstory.com,Trading Story aims to bring NEWER traders prac...,False,id986643659?,0
8232,WSJ Watching Your Wealth,0,"['1412', '26', '1321', '1471']",20,"[6.83, 8.95, 10.2, 8.25, 5.67, 8.68, 6.17, 8.0...",https://itunes.apple.com/us/podcast/wsj-watchi...,http://feeds.wsjonline.com/wsj/podcast_watchin...,http://www.wsj.com/podcasts,Covering all the investment and retirement pla...,False,id1061300303?,0


In [ ]:
final_merge = pd.concat([df_new, new_old])
final_merge

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['Wilderness', 'Podcasts', 'Sports', 'Leisure'...",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed
1,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['History', 'Podcasts', 'Society & Culture']",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill..."
2,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['Music Commentary', 'Podcasts', 'Music']",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ..."
3,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"['Fashion & Beauty', 'Podcasts', 'Arts', 'Heal...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...
4,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"['Comedy Fiction', 'Podcasts', 'Fiction']",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p..."
...,...,...,...,...,...,...,...,...,...,...,...
8235,NaN,"MTG Pro Tutor - Insights, Tips & Advice from M...",https://is1-ssl.mzstatic.com/image/thumb/Podca...,"['1461', '26', '1323', '1460']",Games,229,"[55.63, 48.43, 61.8, 52.75, 43.17, 42.68, 57.5...",https://itunes.apple.com/us/podcast/mtg-pro-tu...,http://mtgprotutor.libsyn.com/rss,http://www.mtgprotutor.com,MTG Pro Tutor is a top rated Magic: The Gather...
8236,NaN,National CASA Podcast,https://is4-ssl.mzstatic.com/image/thumb/Featu...,"['1475', '26', '1325', '1304', '1468']",Government,100,"[25.33, 26.57, 31.58, 2.23, 16.13, 1.72, 1.92,...",https://itunes.apple.com/us/podcast/national-c...,http://feeds.feedburner.com/NationalCasaPodcast,http://www.casaforchildren.org,"Today, more than 1,000 CASA program offices op..."
8237,NaN,CAR DISCOVERY｜PodcastLife,0,"['1454', '26', '1323', '1460', '1324', '1320',...",0,54,"[2.72, 2.72, 2.72, 2.55, 2.55, 2.55, 2.55, 2.5...",https://itunes.apple.com/us/podcast/car-discov...,http://feeds2.feedburner.com/CarDiscovery,http://www.dandl.co.jp/car-discovery/,【www.adelcars.com】アデル・カーズ株式会社提供番組：CAR DISCOVER...
8238,NaN,Tottenham Hotspur Family,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"['1467', '26', '1316']",Sports,112,"[64.53, 60.5, 76.18, 74.55, 88.47, 75.98, 73.5...",https://itunes.apple.com/us/podcast/tottenham-...,http://feeds.soundcloud.com/users/soundcloud:u...,http://soundcloud.com/javad-movahedi-1,The Tottenham Hotspur Family Podcast\n\n\nA gl...


In [ ]:
final_merge.to_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/all_podcast.csv', index = False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/podcast/appledataset/all_podcast.csv')

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description


In [ ]:
import re
urls = df['iTunes URL'].tolist()
for el in urls:
  matches = re.search(r"id\d+", el, re.MULTILINE)
  if matches != None:
    id = matches.group()
    print(int(id[2:]))

Streaming output truncated to the last 5000 lines.
546832353
662443588
1036444741
327763974
509206928
212188040
502414581
399844150
82240478
375450807
168492891
1032020404
1024322894
777268094
1018952191
262254981
498311148
1006743050
973497638
510036484
504897783
273853335
976861197
163354113
369566363
919753766
505494738
904055097
1168702536
258740374
515839372
263032109
120735327
414953489
625985216
430244417
508302878
1032036360
140325086
494450563
82857398
646537599
489840699
1193758599
303923176
950897534
939425867
996367463
994244397
994349813
255293600
1183099838
277285294
645213932
535121941
288726914
1212154379
1151361145
534941251
281777685
1067166096
824963861
105142602
979086248
1025282299
1046085927
887081889
151451644
298292339
1171438277
1086237684
468953819
1019700531
616580985
434012731
313647713
270646795
410954576
1035062584
290478854
964976525
256715878
73330565
1049915804
562296100
177064488
81910746
207700416
668847213
915525189
731355694
419255952
1023213973
217